In [3]:
import sys
import os

# 현재 파일 기준 상위 2단계 폴더 (gnn_with_rwr)를 sys.path에 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [4]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx
from torch_geometric.transforms import RandomLinkSplit
from data.load_dataset import load_cora_graph


In [11]:
def forward(self, x, edge_index):
    print(f"🎯 원본 input x shape: {x.shape}")
    print(f"✅ x[:3]:\n{x[:3]}")

    raw_gate = self.gate[0](x)  # Linear만 적용
    gate_weight = self.gate[1](raw_gate)  # Sigmoid 적용

    print(f"\n🎯 gate linear 출력 (sigmoid 전) shape: {raw_gate.shape}")
    print(f"✅ raw_gate[:3]:\n{raw_gate[:3]}")

    print(f"\n🎯 gate weight (sigmoid 후) shape: {gate_weight.shape}")
    print(f"✅ gate_weight[:3]:\n{gate_weight[:3]}")

    x_gated = x * gate_weight
    print(f"\n🎯 After Gate x shape: {x_gated.shape}")
    print(f"✅ x_gated[:3]:\n{x_gated[:3]}")

    x1 = self.conv1(x_gated, edge_index)
    print(f"\n🎯 After conv1 shape: {x1.shape}")
    print(f"✅ x1[:3]:\n{x1[:3]}")

    x2 = F.relu(x1)
    print(f"\n🎯 After ReLU shape: {x2.shape}")
    print(f"✅ x2[:3]:\n{x2[:3]}")

    x3 = self.conv2(x2, edge_index)
    print(f"\n🎯 After conv2 shape: {x3.shape}")
    print(f"✅ 최종 출력 x3[:3]:\n{x3[:3]}")

    return x3


In [12]:
class GCNEncoderWithGate(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GCNEncoderWithGate, self).__init__()
        self.gate = nn.Sequential(
            nn.Linear(input_dim, input_dim),  # gate[0]
            nn.Sigmoid()                      # gate[1]
        )
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def forward(self, x, edge_index):
        print(f"🎯 원본 input x shape: {x.shape}")
        print(f"✅ x[:3]:\n{x[:3]}")

        raw_gate = self.gate[0](x)
        gate_weight = self.gate[1](raw_gate)

        print(f"\n🎯 gate linear 출력 (sigmoid 전) shape: {raw_gate.shape}")
        print(f"✅ raw_gate[:3]:\n{raw_gate[:3]}")

        print(f"\n🎯 gate weight (sigmoid 후) shape: {gate_weight.shape}")
        print(f"✅ gate_weight[:3]:\n{gate_weight[:3]}")

        x_gated = x * gate_weight
        print(f"\n🎯 After Gate x shape: {x_gated.shape}")
        print(f"✅ x_gated[:3]:\n{x_gated[:3]}")

        x1 = self.conv1(x_gated, edge_index)
        print(f"\n🎯 After conv1 shape: {x1.shape}")
        print(f"✅ x1[:3]:\n{x1[:3]}")

        x2 = F.relu(x1)
        print(f"\n🎯 After ReLU shape: {x2.shape}")
        print(f"✅ x2[:3]:\n{x2[:3]}")

        x3 = self.conv2(x2, edge_index)
        print(f"\n🎯 After conv2 shape: {x3.shape}")
        print(f"✅ 최종 출력 x3[:3]:\n{x3[:3]}")

        return x3


In [13]:
G, id2idx = load_cora_graph()
data = from_networkx(G)
data.train_mask = data.val_mask = data.test_mask = None

transform = RandomLinkSplit(is_undirected=True, add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

In [20]:
attr_path = "/Users/seinkim/bigdas/code/gnn_with_rwr_centrality/attributes/test/"  # 경로 필요시 수정
fname = sorted([f for f in os.listdir(attr_path) if f.endswith(".npy")])[0]
print(f"🔥 실험 파일: {fname}")

attr = np.load(os.path.join(attr_path, fname))
x = torch.tensor(attr, dtype=torch.float)

input_dim = x.size(1)
hidden_dim = input_dim * 3
print(f"✅ input dim: {input_dim}, hidden dim: {hidden_dim}")

🔥 실험 파일: attr_rwr_adasim_simrank_jaccard_degree_top8.npy
✅ input dim: 864, hidden dim: 2592


In [21]:
print(x.shape)

torch.Size([2708, 864])


In [24]:
import numpy as np
rwr = np.load("/Users/seinkim/bigdas/code/gnn_with_rwr_centrality/attributes/generated_2past/attr_rwr_top1.npy")
print("Shape:", rwr.shape)
print("Node 0:", rwr[0])  # 첫 번째 노드의 모든 anchor와의 RWR score

Shape: (2708, 27)
Node 0: [3.38664208e-05 3.55251991e-03 1.09459695e-03 8.90952573e-05
 2.05536421e-04 1.13259756e-04 6.08915580e-05 2.90700939e-05
 6.06193703e-04 6.68578935e-05 1.71766706e-05 8.96565307e-06
 7.37537843e-05 3.07890667e-04 1.72660910e-04 1.55856349e-04
 7.14079826e-05 2.48420771e-05 8.13751367e-06 1.57950218e-05
 2.62721735e-05 9.24306253e-06 2.53794436e-05 1.80008507e-05
 3.32124366e-05 1.08198320e-05 3.80793880e-05]


In [17]:
print(f"✅ x[:3]:\n{x[:3]}")

✅ x[:3]:
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0564, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0251, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0122, 0.0000,
         0.0122, 0.0000, 0.0000, 0.0000, 0.0000, 0.0122, 0.0122, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0122, 0.0000]])
